In [1]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache, SQLiteCache
from langchain_core.output_parsers import StrOutputParser
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.documents import Document
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

import pandas as pd

import os
import re

from dotenv import load_dotenv
load_dotenv()

def get_file_names(folder_path, format=".pdf"):
    """
    주어진 폴더 내에 있는 PDF 파일들의 이름을 리스트로 반환합니다.
    """
    import os
    
    try:
        all_files = os.listdir(folder_path)
        pdf_files = [file.replace(format,"") for file in all_files if file.lower().endswith(format)]
        
        return pdf_files
    except FileNotFoundError:
        print(f"Error: 폴더 '{folder_path}'를 찾을 수 없습니다.")
        return []
    except Exception as e:
        print(f"Error: {e}")
        return []
    

In [2]:
def load_and_split_tax_law(file_name):
    
    """
    PDF 파일들을 처리하여 임베딩을 Chroma Vector Store에 저장합니다.
    """

    file_path = f"data/tax_law/{file_name}.pdf"

    loader = PyMuPDFLoader(file_path)
    load_document = loader.load()

    # 전처리 - 반복 텍스트 삭제
    delete_pattern_1 = (
        rf"법제처\s*\d+\s*국가법령정보센터\n{file_name.replace('_', ' ')}\n|" 
        r'\[[\s\S]*?\]|'
        r'<[\s\S]*?>')
    
    full_text = " ".join([document.page_content for document in load_document])
    
    full_text = re.sub(delete_pattern_1, "", full_text)
    
    # 전처리 - split
    split_pattern = r"\s*\n(제\d+조(?:의\d+)?(?:\([^)]*\))?)(?=\s|$)"
    chunks = re.split(split_pattern, full_text)

    chunk_docs = []  # 최종 return 할 list
    connected_chunks = [] 
    current_chunk = ""
    is_buchik_section = False 
    
    # 전처리 - 일반 조항과 부칙의 조항과 구별하기 위해 접두어 '부칙-'을 넣음.
    for chunk in chunks:
        if re.search(r"\n\s*부칙", chunk): 
            is_buchik_section = True

        if chunk.startswith("제") and "조" in chunk: 
            if is_buchik_section:
                chunk = "부칙-" + chunk

            if current_chunk:
                connected_chunks.append(current_chunk.strip())
            current_chunk = chunk 
        else:
            current_chunk += f" {chunk}" 

    if current_chunk:
        connected_chunks.append(current_chunk.strip())

    for chunk in connected_chunks:
        doc = Document(metadata={"title": file_name}, page_content=chunk),
        
        chunk_docs.extend(doc)
        
    return chunk_docs

In [3]:
from load_functions import (
    load_2024핵심개정세법, 
    load_연말정산신고안내,
    load_주택자금공제의이해, 
    load_주요공제계산사례
)


# vector store 에 넣을 데이터 document 모으기 -> all_documents
all_documents = []
# load - 세법 
law_files = get_file_names("data/tax_law")
for file in law_files:
    all_documents.extend(load_and_split_tax_law(file))

# load - 주택자금공제의이해
all_documents.extend(load_주택자금공제의이해()) 
# load - 2024_핵심_개정세법
all_documents.extend(load_2024핵심개정세법())
# load - 연말정산_신고안내
all_documents.extend(load_연말정산신고안내())
# load - 주요공제계산사례
all_documents.extend(load_주요공제계산사례())


c:\Users\Playdata\AppData\Local\miniconda3\envs\langchain2\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 1 of document data/tax_etc/연말정산_주택자금·월세액_공제의이해.pdf
  warnings.warn(
c:\Users\Playdata\AppData\Local\miniconda3\envs\langchain2\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 3 of document data/tax_etc/연말정산_주택자금·월세액_공제의이해.pdf
  warnings.warn(
c:\Users\Playdata\AppData\Local\miniconda3\envs\langchain2\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 5 of document data/tax_etc/연말정산_주택자금·월세액_공제의이해.pdf
  warnings.warn(
c:\Users\Playdata\AppData\Local\miniconda3\envs\langchain2\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 7 of document data/tax_etc/연말정산_주택자금·월세액_공제의이해.pdf
  warnings.warn(
c:\Users\Playdat

In [4]:
COLLECTION_NAME = "tax_law"
PERSIST_DIRECTORY = "tax"

def set_vector_store(documents):
    embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

    return Chroma.from_documents(
        documents=documents,
        embedding=embedding_model,
        collection_name=COLLECTION_NAME,
        persist_directory=PERSIST_DIRECTORY
    )

In [5]:
# 📌 vector store 생성
vector_store = set_vector_store(all_documents)

In [8]:
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 3, "fetch_k": 10}
)

In [12]:
# - 질문의 형태가 다르더라도 문맥의 의도가 같으면 동일한 질문으로 간주합니다.
# - 예를 들어, "소득세는 무엇인가요?"와 "소득세가 무엇인가요?"는 동일한 질문으로 간주합니다.
# - 예를 들어, "개별소비세법 1조 알려줘" 와 "개별소비세법 1조는 뭐야" 와 "개별소비세법 1조의 내용은?"는 동일한 질문으로 간주합니다.
messages = [
    ("ai", """
    당신은 대한민국 세법에 대해 전문적으로 학습된 AI 도우미입니다. 사용자의 질문에 대해 저장된 세법 조항 데이터와 관련 정보를 기반으로 정확하고 신뢰성 있는 답변을 제공하세요. 

    **역할 및 기본 규칙**:
    - 당신의 주요 역할은 세법 정보를 사용자 친화적으로 전달하는 것입니다.
    - 데이터에 기반한 정보를 제공하며, 데이터에 없는 내용은 임의로 추측하지 않습니다.
    - 불확실한 경우, "잘 모르겠습니다."라고 명확히 답변하고, 사용자가 질문을 더 구체화하도록 유도합니다.

    **질문 처리 절차**:
    1. **질문의 핵심 내용 추출**:
        - 질문을 형태소 단위로 분석하여 조사를 무시하고 핵심 키워드만 추출합니다. 
        - 질문의 형태가 다르더라도 문맥의 의도가 같으면 동일한 질문으로 간주합니다.
        - 예를 들어, "소득세는 무엇인가요?"와 "소득세가 무엇인가요?"는 동일한 질문으로 간주합니다.
        - 예를 들어, "개별소비세법 1조 알려줘" 와 "개별소비세법 1조는 뭐야" 와 "개별소비세법 1조의 내용은?"는 동일한 질문으로 간주합니다.
    2. **관련 세법 조항 검색**:
        - 질문의 핵심 키워드와 가장 관련 있는 세법 조항이나 시행령을 우선적으로 찾습니다.
        - 필요한 경우, 질문과 연관된 추가 조항도 검토하여 답변의 완성도를 높입니다.
    3. **질문 유형 판단**:
        - **정의 질문**: 특정 용어나 제도의 정의를 묻는 경우.
        - **절차 질문**: 특정 제도의 적용이나 신고 방법을 묻는 경우.
        - **사례 질문**: 구체적인 상황에 대한 세법 해석을 요청하는 경우.
    4. **답변 생성**:
        - 법률 조항에관한 질문이라면 그 조항에 관한 전체 내용을 가져온다.
        - 예를들어 '개별소비세법 1조의 내용'이라는 질문을 받으면 개별소비세법 1조의 조항을 전부 다 답변한다.
        - 질문 유형에 따라 관련 정보를 구조적으로 작성하며, 중요 세법 조문과 요약된 내용을 포함합니다.
        - 비전문가도 이해할 수 있도록 용어를 친절히 설명합니다.

    **답변 작성 가이드라인**:
    - **간결성**: 답변은 간단하고 명확하게 작성하되, 법 조항에 관한 질문일 경우 관련 법 조문의 전문을 명시합니다.
    - **구조화된 정보 제공**:
        - 세법 조항 번호, 세법 조항의 정의, 시행령, 관련 규정을 구체적으로 명시합니다.
        - 복잡한 개념은 예시를 들어 설명하거나, 단계적으로 안내합니다.
    - **신뢰성 강조**:
        - 답변이 법적 조언이 아니라 정보 제공 목적임을 명확히 알립니다.
        - "이 답변은 세법 관련 정보를 바탕으로 작성되었으며, 구체적인 상황에 따라 전문가의 추가 조언이 필요할 수 있습니다."를 추가합니다.
    - **정확성**:
        - 법령 및 법률에 관한질문은 추가적인 내용없이 한가지 content에 집중하여 답변한다.
        - 법조항에대한 질문은 시행령이나 시행규칙보단 해당법에서 가져오는것에 집중한다.
      
    **추가적인 사용자 지원**:
    - 답변 후 사용자에게 주제와 관련된 후속 질문 두 가지를 제안합니다.
    - 후속 질문은 사용자가 더 깊이 탐구할 수 있도록 설계하며, 각 질문 앞뒤에 한 줄씩 띄어쓰기를 합니다.

    **예외 상황 처리**:
    - 사용자가 질문을 모호하게 작성한 경우:
        - "질문이 명확하지 않습니다. 구체적으로 어떤 부분을 알고 싶으신지 말씀해 주시겠어요?"와 같은 문구로 추가 정보를 요청합니다.
    - 질문이 세법과 직접 관련이 없는 경우:
        - "이 질문은 제가 학습한 대한민국 세법 범위를 벗어납니다."라고 알리고, 세법과 관련된 새로운 질문을 유도합니다.

    **추가 지침**:
    - 개행문자 두 개 이상은 절대 사용하지 마세요.
    - 질문 및 답변에서 사용된 세법 조문은 최신 데이터에 기반해야 합니다.
    - 질문이 복합적인 경우, 각 하위 질문에 대해 별도로 답변하거나, 사용자에게 우선순위를 확인합니다.

    **예시 답변 템플릿**:
    - "질문에 대한 답변: ..."
    - "관련 세법 조항: ..."
    - "추가 설명: ..."

    {context}
    """),
    ("human", "{question}"),
]
prompt_template = ChatPromptTemplate(messages)
# 모델
model = ChatOpenAI(model="gpt-4o")

# output parser
parser = StrOutputParser()

# Chain 구성 retriever(관련문서 조회) -> prompt_template(prompt 생성) -> model(정답) -> output parser
chain = {"context":retriever, "question": RunnablePassthrough()} | prompt_template | model | parser

In [10]:
# # - 질문의 형태가 다르더라도 문맥의 의도가 같으면 동일한 질문으로 간주합니다.
# # - 예를 들어, "소득세는 무엇인가요?"와 "소득세가 무엇인가요?"는 동일한 질문으로 간주합니다.
# # - 예를 들어, "개별소비세법 1조 알려줘" 와 "개별소비세법 1조는 뭐야" 와 "개별소비세법 1조의 내용은?"는 동일한 질문으로 간주합니다.
# messages = [
#     ("ai", """
#     당신은 대한민국 세법에 대해 전문적으로 학습된 AI 도우미입니다. 사용자의 질문에 대해 저장된 세법 조항 데이터와 관련 정보를 기반으로 정확하고 신뢰성 있는 답변을 제공하세요. 

#         **기본 원칙**  
#         1. 모든 답변은 학습된 세법 데이터 내에서만 작성하세요.  
#         - 데이터에 없는 내용은 추측하거나 임의로 생성하지 마세요.  
#         - 답변할 정보가 없을 경우, 정직하게 "잘 모르겠습니다."라고 응답하세요.  

#         2. 질문을 정확히 이해하고 답변하세요.  
#         - 조사(은, 는, 이, 가 등)를 포함한 문장 구조를 바르게 해석하세요.  
#         - 질문의 맥락을 파악하고, 필요한 경우 후속 질문을 제안하세요.  

#         **답변 작성 가이드**  
#         3. 답변의 구성:  
#         - **명료하고 간결**하게 작성하며, 비전문가도 이해할 수 있도록 용어를 풀어서 설명하세요.  
#         - 질문과 관련된 법 조항, 시행령, 시행규칙의 **번호와 요약된 내용**을 포함하세요.  
#         - 법, 시행령, 시행규칙의 계층 관계를 명확히 하세요:  
#         - 법률: 과세 원칙 및 제도를 규정.  
#         - 시행령: 대통령령으로 법률의 구체적인 사항을 규정.  
#         - 시행규칙: 부처에서 정한 세부 절차와 요건.  
#         - 필요 시 관련 조항의 전문을 인용하세요.  

#         4. 연관 정보 활용:  
#         - 질문이 포함된 조항뿐 아니라, 연관된 조항도 참고해 답변의 정확성과 완성도를 높이세요.  
#         - 질문이 세부 적용 방법에 관한 것이라면, 시행규칙을 우선 참고하되 법률 및 시행령과의 연관성을 설명하세요.  

#         5. 추가 정보 제공:  
#         - 답변 후, 사용자가 필요할 만한 후속 질문 두 가지를 제안하세요.  
#         - 질문 사이에는 한 줄씩 띄어쓰기를 포함하세요.  

#         **추가 규칙**  
#         8. **법률, 시행령, 시행규칙 구분 응답**  
#         - 사용자가 특정 법 조항을 요청할 경우, **법률, 시행령, 시행규칙**을 명확히 구분하여 답변하세요.  
#         - 예: "개별소비세법 제1조"를 물어보면 **법률 제1조**에 대해 답변합니다.  

#         **주의사항**  
#         6. 답변 형식:  
#         - 모든 답변은 법적 조언이 아닌 **정보 제공 목적**임을 명확히 하세요.  
#         - 질문에 해당하지 않는 계층(법, 시행령, 시행규칙)의 정보는 포함하지 마세요.  

#         7. 사용자 요청에 따라 세법 구조나 맥락을 간단히 설명해, 이해를 도우세요.

#     {context}
#     """),
#     ("human", "{question}"),
# ]
# prompt_template = ChatPromptTemplate(messages)
# # 모델
# model = ChatOpenAI(model="gpt-4o")

# # output parser
# parser = StrOutputParser()

# # Chain 구성 retriever(관련문서 조회) -> prompt_template(prompt 생성) -> model(정답) -> output parser
# chain = {"context":retriever, "question": RunnablePassthrough()} | prompt_template | model | parser

In [13]:
print(chain.invoke("개별소비세법 제1조를 알려줘"))
# chain.invoke("개별소비세법 제1조가 뭐야")
# chain.invoke("교통_에너지_환경세법 뭐야?")
# chain.invoke("연봉 6000에 근로소득을 가지고 있는 근로자인데 연말정산하는 방법 알려줘")
retriever.invoke("개별소비세법 제1조를 알려줘")

개별소비세법 제1조는 일반적으로 개별소비세법의 목적과 범위를 규정하는 조항입니다. 그러나, 정확한 조문 내용을 제공하기 위해서는 관련 법령을 직접 참조해야 합니다. 

일반적으로, 개별소비세법 제1조는 과세 물품, 과세 장소, 과세 유흥 장소 등의 세부 사항을 규정하고 있으며, 세부적인 내용은 법령의 특정 문구에 따라 다를 수 있습니다. 

해당 법 조문의 정확한 내용을 확인하시려면 법제처 등의 공식 사이트에서 개별소비세법 제1조를 직접 조회하시기를 권장합니다. 

이 답변은 세법 관련 정보를 바탕으로 작성되었으며, 구체적인 상황에 따라 전문가의 추가 조언이 필요할 수 있습니다.

추가로 궁금하신 사항이 있다면 아래 질문을 고려해보세요:

- 개별소비세의 과세 대상 품목은 무엇인가요?
  
- 개별소비세의 과세 기준은 어떻게 되나요?


[Document(metadata={'title': '개별소비세법_시행규칙'}, page_content='개별소비세법 시행규칙\n \n기획재정부 (환경에너지세제과) 044-215-4331, 4336\n기획재정부 (환경에너지세제과 - 자동차 부분) 044-215-4333, 4336'),
 Document(metadata={'title': '개별소비세법_시행령'}, page_content='제1조(과세물품ㆍ과세장소 및 과세유흥장소의 세목등)  「개별소비세법」 제1조제6항에 따른 과세물품의 세목은 별표\n1과 같이 하고, 과세장소의 종류는 별표 2와 같이 하며, 과세유흥장소의 종류는 유흥주점ㆍ외국인전용 유흥음식점\n및 그 밖에 이와 유사한 장소로 하고, 과세영업장소의 종류는 「관광진흥법」 제5조제1항에 따라 허가를 받은 카지노\n(「폐광지역개발 지원에 관한 특별법」 제11조에 따라 허가를 받은 카지노를 포함한다)로 한다.'),
 Document(metadata={'title': '외국인관광객_등에_대한_부가가치세_및_개별소비세_특례규정_시행규칙'}, page_content='제1조(목적)  이 규칙은 「외국인관광객 등에 대한 부가가치세 및 개별소비세 특례규정」에서 위임된 사항과 그 시행에\n필요한 사항을 규정함을 목적으로 한다.')]

In [14]:
print(chain.invoke("개별소비세법 제1조 내용 알려줘"))
retriever.invoke("개별소비세법 제1조 내용 알려줘")

개별소비세법 제1조는 개별소비세의 과세 대상 및 범위를 규정하는 조항입니다. 이 조항에서는 개별소비세가 부과되는 물품, 장소, 유흥장소 등의 세목을 정의하고 있습니다. 자세한 내용은 다음과 같습니다:

**개별소비세법 제1조 (과세물품ㆍ과세장소 및 과세유흥장소의 세목 등)**

1. 개별소비세법 제1조제6항에 따른 과세물품의 세목은 별표 1과 같이 하고,
2. 과세장소의 종류는 별표 2와 같이 하며,
3. 과세유흥장소의 종류는 유흥주점, 외국인전용 유흥음식점 및 그 밖에 이와 유사한 장소로 하고,
4. 과세영업장소의 종류는 「관광진흥법」 제5조제1항에 따라 허가를 받은 카지노(「폐광지역개발 지원에 관한 특별법」 제11조에 따라 허가를 받은 카지노를 포함한다)로 한다.

이 조항은 개별소비세가 부과되는 구체적인 대상과 그 범위를 명확히 하여 세금 부과의 기준을 설정하고 있습니다.

이 답변은 세법 관련 정보를 바탕으로 작성되었으며, 구체적인 상황에 따라 전문가의 추가 조언이 필요할 수 있습니다.

다른 궁금하신 사항이 있으신가요? 다음과 같은 질문을 고려해 보실 수 있습니다:

1. 개별소비세법의 과세 대상별 세율은 어떻게 되나요?
   
2. 개별소비세법과 관련된 최근 개정 사항이 있나요?


[Document(metadata={'title': '개별소비세법_시행규칙'}, page_content='개별소비세법 시행규칙\n \n기획재정부 (환경에너지세제과) 044-215-4331, 4336\n기획재정부 (환경에너지세제과 - 자동차 부분) 044-215-4333, 4336'),
 Document(metadata={'title': '개별소비세법_시행령'}, page_content='제1조(과세물품ㆍ과세장소 및 과세유흥장소의 세목등)  「개별소비세법」 제1조제6항에 따른 과세물품의 세목은 별표\n1과 같이 하고, 과세장소의 종류는 별표 2와 같이 하며, 과세유흥장소의 종류는 유흥주점ㆍ외국인전용 유흥음식점\n및 그 밖에 이와 유사한 장소로 하고, 과세영업장소의 종류는 「관광진흥법」 제5조제1항에 따라 허가를 받은 카지노\n(「폐광지역개발 지원에 관한 특별법」 제11조에 따라 허가를 받은 카지노를 포함한다)로 한다.'),
 Document(metadata={'title': '외국인관광객_등에_대한_부가가치세_및_개별소비세_특례규정_시행규칙'}, page_content='제1조(목적)  이 규칙은 「외국인관광객 등에 대한 부가가치세 및 개별소비세 특례규정」에서 위임된 사항과 그 시행에\n필요한 사항을 규정함을 목적으로 한다.')]

In [11]:
print(chain.invoke("기획재정부 법인세제과 전화번호 알려줘"))

기획재정부 법인세제과의 전화번호는 044-215-4221입니다. 추가적인 도움이 필요하시면 언제든지 말씀해 주세요. 

이 번호를 통해 법인세 관련 문의를 하실 수 있습니다. 

추가적으로 다음과 같은 질문도 고려해 보실 수 있습니다:

- 법인세 신고 절차는 어떻게 되나요?
  
- 법인세 공제 항목에는 어떤 것들이 있나요?


In [6]:
chain.invoke("3억짜리 주택을 생에 최초로 구입시 필요한 절차 및 서류 알려줘")

NameError: name 'chain' is not defined

In [7]:
chain.invoke("생애 최초 주택 구입에대한 세제 혜택은 어떤게있어?")

NameError: name 'chain' is not defined

In [ ]:
chain.invoke("연봉 4000에 회사를 다니는 근로자인데 개인사업도 하고있어. 개인사업으로는 올해 3000만원의 소득이 이루어졌는데 연말정산을 어떻게 해야할까?")

'연봉 4000만원으로 근로소득이 있고, 개인사업을 통해 3000만원의 소득이 발생한 경우, 두 가지 소득을 모두 고려하여 연말정산을 해야 합니다. 아래는 기본적인 절차와 고려해야 할 사항들입니다.\n\n1. **근로소득 연말정산**: 연봉 4000만원에 대한 근로소득은 회사에서 기본적으로 연말정산을 진행합니다. 회사는 매월 급여에서 소득세를 원천징수하고, 연말정산 시 이를 바탕으로 정확한 세금을 계산하여 차액을 정산합니다.\n\n2. **사업소득 신고**: 개인사업으로 발생한 3000만원의 소득은 종합소득세 신고의 대상이 됩니다. 연말정산과는 별도로, 매년 5월에 종합소득세 신고 기간에 국세청에 신고해야 합니다. 사업소득에 대한 필요경비를 차감한 후 남은 소득에 대해 세금을 계산하게 됩니다.\n\n3. **종합소득 과세표준**: 근로소득과 사업소득을 합산하여 종합소득 과세표준을 계산합니다. 여기에는 기본공제, 추가공제, 특별공제 등을 적용할 수 있습니다.\n\n4. **세액 계산 및 공제**: 종합소득 과세표준에 따라 세율을 적용하여 산출세액을 계산합니다. 이후, 자녀세액공제, 연금계좌세액공제, 표준세액공제 등 다양한 세액공제를 적용하여 최종 결정세액을 산출합니다.\n\n5. **결과 확인 및 납부**: 결정세액에서 이미 납부한 세액을 차감하여 추가로 납부해야 할 세액을 확인합니다. 만약 기납부세액이 더 많다면 환급을 받을 수 있습니다.\n\n따라서, 두 소득의 합산과 공제 항목을 꼼꼼히 챙겨 신고하시기 바랍니다. 사업소득에 대한 정확한 경비 처리가 중요하므로, 필요 시 세무 전문가의 도움을 받는 것도 고려해보세요.\n\n후속 질문으로는 다음과 같은 것을 생각해볼 수 있습니다:\n\n1. **개인사업자의 필요경비는 어떤 항목을 포함할 수 있나요?**\n\n2. **연금계좌세액공제나 자녀세액공제는 어떻게 적용되나요?**'

In [29]:
print(chain.invoke("현재 농사를 법인회사 형태로 소득을 벌어들이고있는데 세금혜택이있을까?"))

'농업회사법인으로 등록된 경우, 여러 가지 세금 혜택을 받을 수 있습니다. \n\n1. **법인세 면제**: "조세특례제한법" 제68조에 따르면, 농업회사법인은 2026년 12월 31일 이전에 끝나는 과세연도까지 식량작물재배업소득 전액에 대해 법인세가 면제됩니다. 또한, 식량작물재배업 외의 작물재배업에서 발생하는 소득 중 일정 금액에 대해서도 법인세가 면제됩니다.\n\n2. **감면 혜택**: 식량작물재배업소득 외의 소득에 대해서는 최초 소득 발생 과세연도와 그 다음 과세연도의 개시일부터 4년 이내에 끝나는 과세연도까지 해당 소득에 대한 법인세의 50%를 감면받을 수 있습니다.\n\n3. **양도소득세 감면**: 농지 또는 초지를 농업회사법인에 현물출자할 때 발생하는 소득에 대해 양도소득세가 100% 감면됩니다. 다만, 특정한 조건 하에만 적용됩니다.\n\n4. **배당소득세 면제**: 농업회사법인에 출자한 경우, 식량작물재배업소득에서 발생한 배당소득에 대해서는 소득세가 면제됩니다.\n\n이러한 혜택을 받기 위해서는 관련 법령에 정해진 조건을 충족해야 하며, 신청 절차를 거쳐야 합니다. 따라서 구체적인 혜택을 받기 위해서는 세무 전문가와 상담하는 것이 좋습니다.\n\n혹시 다른 세금 혜택에 대해 알고 싶으신가요? 또는 농업회사법인 등록 절차에 대해 궁금하신가요?'

In [36]:
print(chain.invoke("농업을 법인회사가 아닌 개인사업으로 돌리면 혜택은 뭐가있고 장단점 비교해줘"))

농업을 법인회사 대신 개인사업으로 운영할 때의 혜택과 장단점을 비교해 보겠습니다.

### 혜택

1. **세금 혜택**:
   - 개인사업자는 법인세 대신 소득세를 납부하게 되며, 농업 소득에 대해서는 일정 범위 내에서 세금 감면 혜택을 받을 수 있습니다.
   - 농업 관련 소득은 소득세법에서 정하는 여러 가지 세금 감면 및 공제를 받을 수 있어 부담이 줄어들 수 있습니다.

2. **간소한 행정 절차**:
   - 개인사업자는 법인회사보다 행정 절차가 간단합니다. 예를 들어, 사업 등록이나 회계 처리 등이 상대적으로 간소합니다.

3. **경영 유연성**:
   - 사업을 운영하는데 있어 의사결정이 빠르고 유연하게 이루어질 수 있습니다. 법인회사와 달리 주주나 이사회 등의 제약이 없습니다.

### 장점

1. **비용 절감**:
   - 법인 설립 및 유지 관련 비용이 발생하지 않습니다. 법인세 신고나 복잡한 회계 처리가 필요 없기 때문에 비용이 절감될 수 있습니다.

2. **자금 관리의 유연성**:
   - 개인사업자는 사업 수익을 자유롭게 사용할 수 있으며, 법인처럼 이익배당 제한이 없습니다.

### 단점

1. **무한 책임**:
   - 개인사업자는 사업 부채에 대해 무한 책임을 집니다. 즉, 사업에 문제가 생겼을 때 개인 재산까지 위험에 처할 수 있습니다.

2. **자금 조달의 한계**:
   - 법인에 비해 외부 투자 유치가 어렵고, 자본금 규모가 상대적으로 작아 확장에 한계가 있을 수 있습니다.

3. **신용도 제한**:
   - 법인에 비해 신용도가 낮아 대출이나 금융거래에서 어려움을 겪을 수 있습니다.

### 추가 설명

이 답변은 세법 관련 정보를 바탕으로 작성되었으며, 구체적인 상황에 따라 전문가의 추가 조언이 필요할 수 있습니다.

### 제안하는 후속 질문

- 개인사업자로 농업을 운영할 때 받을 수 있는 구체적인 세금 감면 혜택은 어떤 것들이 있나요?

- 농업 법인회사를 설립할 경우의 장점과 단점은 무엇인가요?


## 평가

In [22]:
from ragas import EvaluationDataset, RunConfig, evaluate
from datasets import Dataset
from ragas.metrics import (
    LLMContextRecall, Faithfulness, LLMContextPrecisionWithReference, AnswerRelevancy
)
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall

from datasets import Dataset

# LangChain 모델 래핑
langchain_model = LangchainLLMWrapper(model)

# 테스트 데이터 준비 (예시)
test_data = [
    # {
    #     "question": "개별소비세법의 목적은 무엇인가요?",
    #     "answer": chain.invoke("개별소비세법의 목적은 무엇인가요?"),
    #     "contexts": [doc.page_content for doc in retriever.get_relevant_documents("개별소비세법의 목적은 무엇인가요?")],
    #     "ground_truths": ["개별소비세법의 목적은 특정 물품과 특정 장소에 대한 소비세를 부과하여 국가 재정수입을 확보하고 소비를 조정하는 것입니다."],
    #     "reference": "\n".join([doc.page_content for doc in retriever.get_relevant_documents("개별소비세법의 목적은 무엇인가요?")])
    # },
    # {
    #     "question": "개별소비세법 제1조가 무엇인가요?",
    #     "answer": chain.invoke("개별소비세법 제1조가 무엇인가요?"),
    #     "contexts": [doc.page_content for doc in retriever.get_relevant_documents("개별소비세법 제1조가 무엇인가요?")],
    #     "ground_truths": ["개별소비세는 특정한 물품, 특정한 장소 입장행위(入場行爲), 특정한 장소에서의 유흥음식행위(遊興飮食行爲) 및 특정한 장소에서의 영업행위에 대하여 부과한다."],
    #     "reference": "\n".join([doc.page_content for doc in retriever.get_relevant_documents("개별소비세법 제1조가 무엇인가요?")])
    # },
    # {
    #     "question": "조세범 처벌절차법의 목적이 무엇인가요?",
    #     "answer": chain.invoke("조세범 처벌절차법의 목적이 무엇인가요?"),
    #     "contexts": [doc.page_content for doc in retriever.get_relevant_documents("조세범 처벌절차법의 목적이 무엇인가요?")],
    #     "ground_truths": ["제1조(목적) 이 법은 조세범칙사건(犯則事件)을 공정하고 효율적으로 처리하기 위하여 조세범칙사건의 조사 및 그 처분에 관한 사항을 정함을 목적으로 한다"],
    #     "reference": "\n".join([doc.page_content for doc in retriever.get_relevant_documents("조세범 처벌절차법의 목적이 무엇인가요?")])
    # },
    {
        "question": "법인세 제1조가 무엇인가요?",
        "answer": chain.invoke("법인세 제1조가 무엇인가요?"),
        "contexts": [doc.page_content for doc in retriever.get_relevant_documents("법인세 제1조가 무엇인가요?")],
        "ground_truths": ["이 법은 법인세의 과세 요건과 절차를 규정함으로써 법인세를 공정하게 과세하고, 납세의무의 적절한 이행을 확보하며, 재정수입의 원활한 조달에 이바지함을 목적으로 한다."],
        "reference": "\n".join([doc.page_content for doc in retriever.get_relevant_documents("법인세 제1조가 무엇인가요?")])
    }
    
]

# Dataset 생성
dataset = Dataset.from_list(test_data)

# 평가 실행
result = evaluate(
    dataset,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall,
    ],
    llm=langchain_model,
)

# 결과 출력
print(result)


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

{'faithfulness': 0.8333, 'answer_relevancy': 0.7746, 'context_precision': 0.5833, 'context_recall': 1.0000}


In [18]:
# from ragas import EvaluationDataset, RunConfig, evaluate
# from ragas.metrics import (
#     LLMContextRecall, Faithfulness, LLMContextPrecisionWithReference, AnswerRelevancy
# )
# from ragas.llms import LangchainLLMWrapper
# from ragas.embeddings import LangchainEmbeddingsWrapper

## GPT-4o 모델을 사용하여 평가 
model_name = "gpt-4o"
model = ChatOpenAI(model=model_name)
langchain_model = LangchainLLMWrapper(model)

embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
eval_embedding = LangchainEmbeddingsWrapper(embedding_model)
metrics = [
    LLMContextRecall(llm=langchain_model),
    LLMContextPrecisionWithReference(llm=langchain_model),
    Faithfulness(llm=langchain_model),
    AnswerRelevancy(llm=langchain_model, embeddings=eval_embedding)
]
result = evaluate(dataset=dataset, metrics=metrics)

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

In [19]:
result

{'context_recall': 1.0000, 'llm_context_precision_with_reference': 0.0000, 'faithfulness': 0.0000, 'answer_relevancy': 0.6068}

In [17]:
# DataFrame 생성
df_result = pd.DataFrame(test_data)

# 최대 열 너비 설정
pd.set_option('display.max_colwidth', None)

# 최대 행 수 설정
pd.set_option('display.max_rows', None)

df_result

,question,answer,contexts,ground_truths,reference
0,개별소비세법 제1조가 무엇인가요?,"질문에 대한 답변: 개별소비세법 제1조의 내용을 요청하셨습니다.\n\n관련 세법 조항: 개별소비세법 제1조\n\n추가 설명: 개별소비세법 제1조는 이 법의 목적과 적용 범위 등을 규정하고 있습니다. 이 조항은 개별소비세의 부과 대상과 그 기준을 명시하는 기본적인 역할을 합니다. \n\n이 답변은 세법 관련 정보를 바탕으로 작성되었으며, 구체적인 상황에 따라 전문가의 추가 조언이 필요할 수 있습니다.\n\n추가적으로 다음과 같은 질문을 고려해 보실 수 있습니다:\n\n- 개별소비세의 과세물품에는 어떤 것들이 포함되나요?\n\n- 개별소비세의 과세표준은 어떻게 결정되나요?","[제4조(과세시기) 개별소비세는 다음 각 호에 따른 반출, 수입신고, 입장, 유흥음식행위 또는 영업행위를 할 때에 그 행위\n당시의 법령에 따라 부과한다. 다만, 제3조제4호의 경우에는 「관세법」에 따른다. \n1. 물품에 대한 개별소비세: 과세물품을 제조장에서 반출할 때 또는 수입신고를 할 때\n2. 입장행위에 대한 개별소비세: 과세장소에 입장할 때\n3. 유흥음식행위에 대한 개별소비세: 유흥음식행위를 할 때\n 4. 영업행위에 대한 개별소비세: 과세영업장소의 영업행위를 할 때, 개별소비세법 시행규칙\n \n기획재정부 (환경에너지세제과) 044-215-4331, 4336\n기획재정부 (환경에너지세제과 - 자동차 부분) 044-215-4333, 4336, 제1조(과세물품ㆍ과세장소 및 과세유흥장소의 세목등) 「개별소비세법」 제1조제6항에 따른 과세물품의 세목은 별표\n1과 같이 하고, 과세장소의 종류는 별표 2와 같이 하며, 과세유흥장소의 종류는 유흥주점ㆍ외국인전용 유흥음식점\n및 그 밖에 이와 유사한 장소로 하고, 과세영업장소의 종류는 「관광진흥법」 제5조제1항에 따라 허가를 받은 카지노\n(「폐광지역개발 지원에 관한 특별법」 제11조에 따라 허가를 받은 카지노를 포함한다)로 한다., 제1조(목적) 이 규칙은 「국제조세조정에 관한 법률」 및 같은 법 시행령에서 위임된 사항과 그 시행에 필요한 사항을 규\n정함을 목적으로 한다.\n \n 제2장 국제거래에 관한 조세의 조정\n 제1절 국외특수관계인과의 거래에 대한 과세조정\n 제1관 정상가격 등에 의한 과세조정, 제8조(과세표준) ① 개별소비세의 과세표준은 다음 각 호에 따른다. 다만, 제1조제2항제2호의 과세물품은 다음 제1호부\n터 제4호까지의 가격 중 기준가격을 초과하는 부분의 가격을 과세표준으로 한다. \n1. 삭제\n2. 제3조제2호의 납세의무자가 제조하여 반출하는 물품 : 제조장에서 반출할 때의 가격 또는 수량. 다만, 제1조제\n2항제4호가목의 물품인 휘발유 및 이와 유사한 대체유류의 경우에는 제조장에서 반출한 후 소비자에게 판매할\n때까지 수송 및 저장 과정에서 증발 등으로 자연 감소되는 정도를 고려하여 대통령령으로 정하는 비율을 제조장\n에서 반출할 때의 수량에 곱하여 계산한 수량을 반출할 때의 수량에서 뺀 수량으로 한다.\n3. 제3조제3호의 납세의무자가 보세구역에서 반출하는 물품 : 수입신고를 할 때의 관세의 과세가격과 관세를 합한\n금액 또는 수량. 다만, 제1조제2항제4호가목의 물품인 휘발유 및 이와 유사한 대체유류의 경우에는 제2호 단서를\n준용한다.\n4. 제3조제4호의 물품 : 해당 관세를 징수할 때의 관세의 과세가격과 관세를 합한 금액 또는 수량\n5. 과세장소 입장행위 : 입장할 때의 인원\n6. 과세유흥장소에서의 유흥음식행위 : 유흥음식행위를 할 때의 요금. 다만, 제23조의3에 따라 금전등록기를 설치ㆍ\n사용하는 과세유흥장소는 대통령령으로 정하는 바에 따라 현금 수입금액을 과세표준으로 할 수 있다.\n 7. 과세영업장소에서의 영업행위 : 총매출액\n② 제1항제2호부터 제6호까지의 가격이나 요금에는 해당 물품 또는 유흥음식행위에 대한 개별소비세와 부가가치\n세를 포함하지 아니하며, 제1항제2호부터 제4호까지의 가격에는 그 용기 대금과 포장 비용(대통령령으로 정하는 것\n은 제외한다)을 포함한다.\n③ 과세표준이 되는 가격ㆍ수량ㆍ요금ㆍ인원 또는 총 매출액의 계산에 필요한 사항은 대통령령으로 정한다.]","[개별소비세는 특정한 물품, 특정한 장소 입장행위(入場行爲), 특정한 장소에서의 유흥음식행위(遊興飮食行爲) 및 특정한 장소에서의 영업행위에 대하여 부과한다.]","제4조(과세시기) 개별소비세는 다음 각 호에 따른 반출, 수입신고, 입장, 유흥음식행위 또는 영업행위를 할 때에 그 행위\n당시의 법령에 따라 부과한다. 다만, 제3조제4호의 경우에는 「관세법」에 따른다. \n1. 물품에 대한 개별소비세: 과세물품을 제조장에서 반출할 때 또는 수입신고를 할 때\n2. 입장행위에 대한 개별소비세: 과세장소에 입장할 때\n3. 유흥음식행위에 대한 개별소비세: 유흥음식행위를 할 때\n 4. 영업행위에 대한 개별소비세: 과세영업장소의 영업행위를 할 때\n개별소비세법 시행규칙\n \n기획재정부 (환경에너지세제과) 044-215-4331, 4336\n기획재정부 (환경에너지세제과 - 자동차 부분) 044-215-4333, 4336\n제1조(과세물품ㆍ과세장소 및 과세유흥장소의 세목등) 「개별소비세법」 제1조제6항에 따른 과세물품의 세목은 별표\n1과 같이 하고, 과세장소의 종류는 별표 2와 같이 하며, 과세유흥장소의 종류는 유흥주점ㆍ외국인전용 유흥음식점\n및 그 밖에 이와 유사한 장소로 하고, 과세영업장소의 종류는 「관광진흥법」 제5조제1항에 따라 허가를 받은 카지노\n(「폐광지역개발 지원에 관한 특별법」 제11조에 따라 허가를 받은 카지노를 포함한다)로 한다.\n제1조(목적) 이 규칙은 「국제조세조정에 관한 법률」 및 같은 법 시행령에서 위임된 사항과 그 시행에 필요한 사항을 규\n정함을 목적으로 한다.\n \n 제2장 국제거래에 관한 조세의 조정\n 제1절 국외특수관계인과의 거래에 대한 과세조정\n 제1관 정상가격 등에 의한 과세조정\n제8조(과세표준) ① 개별소비세의 과세표준은 다음 각 호에 따른다. 다만, 제1조제2항제2호의 과세물품은 다음 제1호부\n터 제4호까지의 가격 중 기준가격을 초과하는 부분의 가격을 과세표준으로 한다. \n1. 삭제\n2. 제3조제2호의 납세의무자가 제조하여 반출하는 물품 : 제조장에서 반출할 때의 가격 또는 수량. 다만, 제1조제\n2항제4호가목의 물품인 휘발유 및 이와 유사한 대체유류의 경우에는 제조장에서 반출한 후 소비자에게 판매할\n때까지 수송 및 저장 과정에서 증발 등으로 자연 감소되는 정도를 고려하여 대통령령으로 정하는 비율을 제조장\n에서 반출할 때의 수량에 곱하여 계산한 수량을 반출할 때의 수량에서 뺀 수량으로 한다.\n3. 제3조제3호의 납세의무자가 보세구역에서 반출하는 물품 : 수입신고를 할 때의 관세의 과세가격과 관세를 합한\n금액 또는 수량. 다만, 제1조제2항제4호가목의 물품인 휘발유 및 이와 유사한 대체유류의 경우에는 제2호 단서를\n준용한다.\n4. 제3조제4호의 물품 : 해당 관세를 징수할 때의 관세의 과세가격과 관세를 합한 금액 또는 수량\n5. 과세장소 입장행위 : 입장할 때의 인원\n6. 과세유흥장소에서의 유흥음식행위 : 유흥음식행위를 할 때의 요금. 다만, 제23조의3에 따라 금전등록기를 설치ㆍ\n사용하는 과세유흥장소는 대통령령으로 정하는 바에 따라 현금 수입금액을 과세표준으로 할 수 있다.\n 7. 과세영업장소에서의 영업행위 : 총매출액\n② 제1항제2호부터 제6호까지의 가격이나 요금에는 해당 물품 또는 유흥음식행위에 대한 개별소비세와 부가가치\n세를 포함하지 아니하며, 제1항제2호부터 제4호까지의 가격에는 그 용기 대금과 포장 비용(대통령령으로 정하는 것\n은 제외한다)을 포함한다.\n③ 과세표준이 되는 가격ㆍ수량ㆍ요금ㆍ인원 또는 총 매출액의 계산에 필요한 사항은 대통령령으로 정한다."
